In [19]:
import numpy.random as random
import numpy as np
import pandas as pd
import matplotlib.pylab as plt


In [132]:
def generate_data(n=100, mA=[1.0, 0.5], mB=[-1.0, 0.0], sigmaA=0.5, sigmaB=0.5):
    classA = np.vstack((random.randn(1, n) * sigmaA + mA[0],
                        random.randn(1, n) * sigmaA + mA[1],
                        ))
    y_true_A = np.ones((1, n))

    classB = np.vstack((random.randn(1, n) * sigmaB + mB[0],
                        random.randn(1, n) * sigmaB + mB[1],
                        ))
    y_true_B = -np.ones((1, n))

    data = np.concatenate((classA, classB), axis=1)
    y_true = np.concatenate((y_true_A, y_true_B), axis=1)

    data_with_bias = np.vstack((data, np.ones((1, 2 * n))))

    return data_with_bias, y_true


data_with_bias, y_true = generate_data()

display(pd.DataFrame(data_with_bias))
display(pd.DataFrame(y_true))

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,1.126996,1.500600,1.206742,2.074139,0.755189,0.888092,1.882931,0.281237,1.244934,0.478274,...,-0.541346,-1.095500,-1.241513,-1.896676,-0.971841,-0.676310,-1.801913,-1.599174,-1.477937,-1.170900
1,0.213762,0.905504,-0.634677,0.181431,0.750798,0.580269,1.097511,0.349046,0.905629,1.135401,...,-0.203722,0.438203,-0.396858,0.462341,-0.063715,-0.229065,-0.454522,-0.182546,-0.302757,0.318321
2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [133]:
def forward_pass(P, W, V, n_data):
  #X = np.vstack((P, np.ones((1, n_data))))

  H_in = W @ P
  H_out = np.vstack((2 / (1 + np.exp(-H_in) )- 1, np.ones((1, n_data))))

  O_in = V @ H_out
  O_out  = 2 / (1 + np.exp(-O_in)) - 1

  return O_out, H_out

def backward_pass(O, H, T, V, n_hidden):
  delta_O = (O - T) * ((1 + O) - (1 - O)) * 0.5

  delta_H = (V.T * delta_O) * ((1 + H) * (1 - H)) * 0.5
  delta_H = delta_H[1:n_hidden+1, :]

  return delta_O, delta_H  

def update_weights(X, H, d_H, d_O, d_W, d_V, alpha, eta):
  d_W = (d_W * alpha) - (d_H @ X.T) * (1 -alpha)
  d_V = (d_V * alpha) - (d_O @ H.T) * (1 -alpha)

  return d_W, d_V

def gen_delta(X, T, n_data, n_epoch, n_hidden, alpha, eta):
  #Init the weight matrixes
  W = np.ones((n_hidden, 3))
  V = np.ones((1, n_hidden+1))

  #Init the output and hidden layer matrices 
  O, H = [] , []
  d_W, d_V = np.zeros((n_hidden, 3)), np.zeros((1, n_hidden+1))

  #Perform the number of epoch requested
  for i in range(0, n_epoch) :
    O , H = forward_pass(X, W, V, n_data)
    delta_O , delta_H = backward_pass(O, H, T, V, n_hidden)
    d_W, d_V = update_weights(X, H, delta_H, delta_O, d_W, d_V, alpha, eta)

    W = W + d_W*eta
    V = V + d_V*eta

  return O  

In [134]:
#Testing the backprop on a generated data batch
X , classes = generate_data()

W = np.ones((3, 3))
V= np.ones((1,4))

O = gen_delta(X, classes, 200, 5000, 4, 0.1, 0.01)

display(pd.DataFrame(O))
display(pd.DataFrame(classes))

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,1.0,1.0,1.0,0.999765,0.85606,1.0,1.0,1.0,1.0,1.0,...,0.029217,-0.03303,-0.013583,-0.022142,-0.038089,0.050605,-0.070437,-0.002714,-0.002612,0.0114


,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
